In [83]:
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
!pip install mpld3
import mpld3
mpld3.enable_notebook()

In [84]:
def plotseird(t, colors, **kwargs):
  f, ax = plt.subplots(1,1,figsize=(10, 6))

  for metric_name, metric_values in kwargs.items():
    color = colors.get(metric_name, 'b')
    ax.plot(t, metric_values, color, alpha=0.8, linewidth=2.5, label=metric_name)

  ax.set_xlabel('Time (days)', labelpad = 12)
  # ax.set_ylabel('Population', labelpad = 35)
  ax.yaxis.set_tick_params(length=0)
  ax.xaxis.set_tick_params(length=0)
  ax.grid(b=True, which='major', c='w', lw=2, ls='-')
  legend = ax.legend(borderpad=2.0)
  legend.get_frame().set_alpha(0.6)
  for spine in ('top', 'right', 'bottom', 'left'):
      ax.spines[spine].set_visible(False)
  plt.savefig('last.png')
  plt.show()

In [85]:
def deriv(y, t, betta0, gamma, N0, fi, g, lmb, T, k, eps, alph, test):
    S, I, R, D, M, P = y
    
    dPdt = g * D - lmb * P                        #2.1f
    
    betta = betta0 * (test ** (-eps * T)) * abs(1+alph) * abs(1 - (dPdt/N0) ** k)
    #убрал H
    dSdt = (-betta / N0) * I * S               #2.1a
    dIdt = (betta / N0) * S * I - (gamma * I)   #2.1b
    dRdt = (1 - fi) * I * gamma                   #2.1c
    dDdt = fi * gamma * I - g * D                 #2.1d
    dMdt = g * D                                  #2.1e
    
    N = dSdt + dIdt + dRdt + dDdt + dMdt
    return betta, dSdt, dIdt, dRdt, dDdt, dMdt, dPdt, N

In [86]:
def get_report(deriv, S0, betta0, gamma, N0, fi, g, lmb, T, k, eps, alph, test, I0, R0, D0, M0, P0):
  t = np.linspace(0, 364, 365) # Grid of time points (in days)
  y0 = S0, I0, R0, D0, M0, P0

  # Integrate the SIR equations over the time grid, t.
  ret = odeint(deriv, y0, t, args=(betta0, gamma, N0, fi, g, lmb, T, k, eps, alph, test))
  S, I, R, D, M, P = ret.T

  COLORS_MAP = {
    "Susceptible" : '-.b',
    "Exposed": '--c',
    "Infected": 'r',
    "Recovered": '--g',
    "Cumulative": 'm',
    "Population": "violet",
    "Dead": "-.k"
  }
  plotseird(t, colors = COLORS_MAP, Exposed = E, Infected = I, Dead = D)
  plotseird(t, colors = COLORS_MAP, Infected = I, Recovered = R, Susceptible = S)

In [87]:
parameters = dict(
    betta0 = 1,              #Базовая скорость передачи
    gamma = 4 / 8.,         #Cредний инфекционный период
    N0 = 130000,            #Общее число населения
    fi = 0.0043,            #доля смертности
    g = 1,                  #
    lmb = 0.1,                 #Cредняя продолжительность общественной реакции
    T = 0.1,                   #Температура
    k = 695.1,              #Скорость реагирования
    eps = 0.016,            #Интенсивность влияния изменения температуры.
    alph = 0.130,           #сила правительственных действий
    test = 1, #e
    
    S0 = 130000,    #initial susceptible population
    I0 = 0.00025,                 # Initial Exposed
    R0 = 0,                 # Initial Infectious
    D0 = 0,                 # Initial Removed
    M0 = 0,                 # Inital Dead
    P0 = 0                  # Initial Cumulative cases
    
)
get_report(deriv, **parameters)

RuntimeError: The size of the array returned by func (8) does not match the size of y0 (6).